<center><p float="center">
  <img src="https://upload.wikimedia.org/wikipedia/commons/e/e9/4_RGB_McCombs_School_Brand_Branded.png" width="300" height="100"/>
  <img src="https://mma.prnewswire.com/media/1458111/Great_Learning_Logo.jpg?p=facebook" width="200" height="100"/>
</p></center>

<h1><center><font size=10>Artificial Intelligence and Machine Learning</center></font></h1>
<h1><center>Introduction to Natural Language Processing - Week 4</center></h1>

<center><p float="center">
  <img src="https://images.pexels.com/photos/4185957/pexels-photo-4185957.jpeg?auto=compress&cs=tinysrgb&w=1260&h=750&dpr=1" width="720"/>
</p></center>

<center><font size=6>News Article Categorization and Summarization</font></center>

## Problem Statement

### Business Context

In the dynamic landscape of the media and news industry, the ability to swiftly categorize and curate content has become a strategic imperative. The vast volume of information demands efficient systems to organize and present content to the audience.

The media industry, being the pulse of information dissemination, grapples with the continuous influx of news articles spanning diverse topics. Ensuring that the right articles reach the right audience promptly is not just a logistical necessity but a critical component in retaining and engaging audiences in an age of information overload.

Common Industry Challenges:
Amidst the ceaseless flow of news, organizations encounter challenges such as:
- Information Overload: The sheer volume of news articles makes manual categorization impractical.
- Timeliness: Delays in categorizing news articles can result in outdated or misplaced content.

### Problem Definition

E-news Express, a news aggregation startup, faces the challenge of categorizing the news articles collected. With news articles covering sports, entertainment, politics, and more, the need for an advanced and automated system to categorize them has become increasingly evident. The manual efforts required for categorizing such a diverse range of news articles are substantial, and human errors in the categorization of news articles can lead to reputational damage for the startup. There is also the factor of delays and potential inaccuracies. To streamline and optimize this process, the organization recognizes the imperative of adopting cutting-edge technologies, particularly machine learning, to automate and enhance the categorization of content.

As a data scientist on the E-news Express data team, the task is to analyze the text in news articles and build an unsupervised learning model for categorizing them. The categorization will be followed up by a summarization of the news article so that users can read through the summary first and then click on a relevant link to dive in deeper if needed. The goal is to optimize the categorization and summarization process, ensuring timely and personalized delivery.

### Data Dictionary

- **Article**: The main body of the news article

## Installing and Importing Necessary Libraries

In [ ]:
# Installation for GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python --force-reinstall --upgrade --no-cache-dir -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 111.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 242.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 144.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.2/133.2 kB 317.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which 

In [ ]:
# For downloading the models from HF Hub
!pip install huggingface_hub datasets -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.7 MB/s eta 0:00:00


In [ ]:
# Importing library for data manipulation
import pandas as pd

# Function to download the model from the Hugging Face model hub
from huggingface_hub import hf_hub_download

# Importing the Llama class from the llama_cpp module
from llama_cpp import Llama

# Importing the json library
import json

#Importing the datasets library from huggingface
from datasets import load_dataset

## Load the dataset

In [ ]:
data = pd.read_csv("articles.csv")

## Data Overview

In [ ]:
#Printing the first few rows.
data.head()

,Article
0,A New Push to Loosen New York's Divorce Law La...
1,Memos Warned of Billing Fraud by Firm in Iraq ...
2,"New Chechen Leader Vows Peace, Poll Criticized..."
3,WWW KOTV.com _ Two of three UN workers kidnapp...
4,U.S. State Dept Finishes Review of Iraq Aid Pl...


In [ ]:
#Number of news articles.
print(len(data))

100


## Model Building

### Loading the model

In [ ]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGUF"
model_basename = "llama-2-13b-chat.Q5_K_M.gguf" # the model is in gguf format

In [ ]:
# Using hf_hub_download to download a model from the Hugging Face model hub
# The repo_id parameter specifies the model name or path in the Hugging Face repository
# The filename parameter specifies the name of the file to download
model_path = hf_hub_download(
    repo_id=model_name_or_path,
    filename=model_basename
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


llama-2-13b-chat.Q5_K_M.gguf:   0%|          | 0.00/9.23G [00:00<?, ?B/s]

In [ ]:
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2,  # CPU cores
    n_batch=512,  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=43,  # Change this value based on your model and your GPU VRAM pool.
    n_ctx=4096,  # Context window
)

AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 
Model metadata: {'tokenizer.ggml.unknown_token_id': '0', 'tokenizer.ggml.eos_token_id': '2', 'general.architecture': 'llama', 'llama.context_length': '4096', 'general.name': 'LLaMA v2', 'llama.embedding_length': '5120', 'llama.feed_forward_length': '13824', 'llama.attention.layer_norm_rms_epsilon': '0.000010', 'llama.rope.dimension_count': '128', 'llama.attention.head_count': '40', 'tokenizer.ggml.bos_token_id': '1', 'llama.block_count': '40', 'llama.attention.head_count_kv': '40', 'general.quantization_version': '2', 'tokenizer.ggml.model': 'llama', 'general.file_type': '17'}


### Defining Model Response Parameters

In [ ]:
def generate_llama_response(instruction, review):

    # System message explicitly instructing not to include the review text
    system_message = """
        [INST]<<SYS>>
        {}
        <</SYS>>[/INST]
    """.format(instruction)

    # Combine user_prompt and system_message to create the prompt
    prompt = f"{review}\n{system_message}"

    # Generate a response from the LLaMA model
    response = lcpp_llm(
        prompt=prompt,
        max_tokens=1024,
        temperature=0,
        top_p=0.95,
        repeat_penalty=1.2,
        top_k=50,
        stop=['INST'],
        echo=False,
        seed=42,
    )

    # Extract the sentiment from the response
    response_text = response["choices"][0]["text"]
    return response_text

- **`max_tokens`**: This parameter **specifies the maximum number of tokens that the model should generate** in response to the prompt.

- **`temperature`**: This parameter **controls the randomness of the generated response**. A higher temperature value will result in a more random response, while a lower temperature value will result in a more predictable response.

- **`top_p`**: This parameter **controls the diversity of the generated response by establishing a cumulative probability cutoff for token selection**. A higher value of top_p will result in a more diverse response, while a lower value will result in a less diverse response.

- **`repeat_penalty`**: This parameter **controls the penalty for repeating tokens in the generated response**. A higher value of repeat_penalty will result in a lower probability of repeating tokens, while a lower value will result in a higher probability of repeating tokens.

- **`top_k`**: This parameter **controls the maximum number of most-likely next tokens to consider** when generating the response at each step.

- **`stop`**: This parameter is a **list of tokens that are used to dynamically stop response generation** whenever the tokens in the list are encountered.

- **`echo`**: This parameter **controls whether the input (prompt) to the model should be returned** in the model response.

- **`seed`**: This parameter **specifies a seed value that helps replicate results**.


## Classifying the news articles

In [ ]:
data_1 = data.copy()

In [ ]:
# defining the instructions for the model
instruction_1 = """
    You are an AI analyzing news articles. Tag the given article using one or more of the below mentioned categories only depending upon the content of the article:
    - World
    - Sports
    - Business
    - Sci/Tech
"""

In [ ]:
data_1['model_response_1'] = data_1['Article'].apply(lambda x: generate_llama_response(instruction_1, x))

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.gene

In [ ]:
#Printing the model's response.
data_1["model_response_1"]

0      Sure! Here's the article you provided, tagged...
1      Sure, I can help you with that! Here's the ar...
2      Sure! Here's the article you provided, tagged...
3      Sure, I can help you with that! Based on the ...
4      Sure, I can help you with that! Here's the ar...
                            ...                        
95     Sure! Here's the article you provided, tagged...
96     Sure, I can help you with that! Here's the ar...
97     Sure! Here's the article you provided, tagged...
98     Sure, I can help you with that! Here's the ar...
99     Sure! Here's the article you provided, tagged...
Name: model_response_1, Length: 100, dtype: object

In [ ]:
#Printing the model's response.
data_1["model_response_1"][0]

" Sure! Here's the article you provided, tagged according to its content:\n\nArticle: A New Push to Loosen New York's Divorce Law\n\nTags:\n\n* Business (lawyers and judges pushing for change in divorce law)"

In [ ]:
def extract_label(model_response):
    if 'business' in model_response.lower():
        return 'Business'
    elif 'world' in model_response.lower():
        return 'World'
    elif 'sports' in model_response.lower():
        return 'Sports'
    else:
      return 'Sci/Tech'

In [ ]:
data_1["Label"] = data_1["model_response_1"].apply(extract_label)

In [ ]:
final_data_1 = data_1.drop(['model_response_1'], axis=1)
final_data_1.head()

,Article,Label
0,A New Push to Loosen New York's Divorce Law La...,Business
1,Memos Warned of Billing Fraud by Firm in Iraq ...,Business
2,"New Chechen Leader Vows Peace, Poll Criticized...",World
3,WWW KOTV.com _ Two of three UN workers kidnapp...,Business
4,U.S. State Dept Finishes Review of Iraq Aid Pl...,World


## Classifying the news article and returning a structured output

In [ ]:
data_2 = data.copy()

In [ ]:
# defining the instructions for the model
instruction_2 = """
    You are an AI analyzing news articles. Tag the given article using one or more of the below mentioned categories only and not any other depending upon the content of the article:
    - World
    - Sports
    - Business
    - Sci/Tech

    Format the output as a JSON object with a single key-value pair as shown below:
    {"label": "your_label_prediction"}
"""

In [ ]:
data_2['model_response_2'] = data_2['Article'].apply(lambda x: generate_llama_response(instruction_2, x))

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.gene

In [ ]:
#Printing the model's response.
data_2["model_response_2"][0]

' Sure, I can help you with that! Based on the content of the article you provided, I would tag it under the category of:\n\n{"label": "Business"}\n\nReasoning: The article is discussing a push to change New York\'s divorce law, which falls under the category of business or legal news.'

In [ ]:
# defining a function to parse the JSON output from the model
def extract_json_data(json_str):
    try:
        # Find the indices of the opening and closing curly braces
        json_start = json_str.find('{')
        json_end = json_str.rfind('}')

        if json_start != -1 and json_end != -1:
            extracted_sentiment = json_str[json_start:json_end + 1]  # Extract the JSON object
            data_dict = json.loads(extracted_sentiment)
            return data_dict
        else:
            print(f"Warning: JSON object not found in response: {json_str}")
            return {}
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON: {e}")
        return {}

In [ ]:
#Applying the function.
data_2["model_response_2_parsed"] = data_2["model_response_2"].apply(extract_json_data)

In [ ]:
#Printing the labels predicted by the model.
data_2["model_response_2_parsed"]

0           {'label': 'Business'}
1           {'label': 'Business'}
2              {'label': 'World'}
3              {'label': 'World'}
4              {'label': 'World'}
                 ...             
95    {'label': 'Atomic Imaging'}
96          {'label': 'Sci/Tech'}
97          {'label': 'Sci/Tech'}
98          {'label': 'Business'}
99          {'label': 'Business'}
Name: model_response_2_parsed, Length: 100, dtype: object

In [ ]:
data_2[data_2["model_response_2_parsed"]=={}]

,Article,model_response_2,model_response_2_parsed


In [ ]:
model_response_parsed_df_2 = pd.json_normalize(data_2['model_response_2_parsed'])
model_response_parsed_df_2.head()

,label
0,Business
1,Business
2,World
3,World
4,World


In [ ]:
data_with_parsed_model_output_2 = pd.concat([data_2, model_response_parsed_df_2], axis=1)
data_with_parsed_model_output_2.head()

,Article,model_response_2,model_response_2_parsed,label
0,A New Push to Loosen New York's Divorce Law La...,"Sure, I can help you with that! Based on the ...",{'label': 'Business'},Business
1,Memos Warned of Billing Fraud by Firm in Iraq ...,Sure! Here's my analysis of the given article...,{'label': 'Business'},Business
2,"New Chechen Leader Vows Peace, Poll Criticized...","Sure, I can help you with that! Based on the ...",{'label': 'World'},World
3,WWW KOTV.com _ Two of three UN workers kidnapp...,"Sure, I can help you with that! Based on the ...",{'label': 'World'},World
4,U.S. State Dept Finishes Review of Iraq Aid Pl...,"Sure, I can help you with that! Based on the ...",{'label': 'World'},World


In [ ]:
final_data_2 = data_with_parsed_model_output_2.drop(['model_response_2','model_response_2_parsed'], axis=1)
final_data_2.head()

,Article,label
0,A New Push to Loosen New York's Divorce Law La...,Business
1,Memos Warned of Billing Fraud by Firm in Iraq ...,Business
2,"New Chechen Leader Vows Peace, Poll Criticized...",World
3,WWW KOTV.com _ Two of three UN workers kidnapp...,World
4,U.S. State Dept Finishes Review of Iraq Aid Pl...,World


## Classifying the news articles and generating a headline

In [ ]:
data_3 = data.copy()

In [ ]:
# defining the instructions for the model
instruction_3 = """
    You are an AI analyzing news articles. Tag the given article using one or more of the below mentioned categories only depending upon the content of the article:
    - World
    - Sports
    - Business
    - Sci/Tech

    Once done,craft a compelling headline summarizing the key insights and significance of the article.Ensure the headline is attention-grabbing and concise while accurately reflecting the content and output it.

    Provide the output in a JSON format with the following keys:
    {
        "Label": "your_label_prediction",
        "Headline": "your_headline_prediction",
    }

    Only return the JSON, do not return any other information and remove the extra spaces.
"""

In [ ]:
data_3['model_response_3'] = data_3['Article'].apply(lambda x: generate_llama_response(instruction_3, x).replace('\n', ''))

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.gene

In [ ]:
data_3["model_response_3_parsed"] = data_3["model_response_3"].apply(extract_json_data)

Error parsing JSON: Invalid \escape: line 1 column 98 (char 97)


In [ ]:
data_3["model_response_3_parsed"]

0     {'Label': 'Business', 'Headline': 'New York Di...
1     {'Label': 'Business', 'Headline': 'Memos Warne...
2     {'Label': 'World', 'Headline': 'New Chechen Le...
3     {'Label': 'World', 'Headline': 'Two UN Workers...
4     {'Label': 'World', 'Headline': 'U.S. State Dep...
                            ...                        
95    {'Label': 'Sci/Tech', 'Headline': 'Atomic Visi...
96    {'Label': 'Sci/Tech', 'Headline': 'Ninth-grade...
97    {'Label': 'Politics', 'Headline': 'Congress Wa...
98    {'Label': 'Business', 'Headline': 'IBM Jumps i...
99    {'Label': 'Business', 'Headline': 'Clearing Th...
Name: model_response_3_parsed, Length: 100, dtype: object

In [ ]:
data_3[data_3["model_response_3_parsed"]=={}]

,Article,model_response_3,model_response_3_parsed
43,Hewitt wins Washington Open Australia #39;s Ll...,"{""Label"": ""Sports"",""Headline"": ""Hewitt wins Wa...",{}


In [ ]:
data_3["model_response_3"][43]

'{"Label": "Sports","Headline": "Hewitt wins Washington Open: Australia\'s Lleyton Hewitt Captures \\$US500,000 Tournament"}'

In [ ]:
data_3["model_response_3_parsed"][43] = {"label":"Sports","Headline":"Hewitt wins Washington Open: Australia's Lleyton Hewitt Captures $US500,000 Tournament"}

In [ ]:
model_response_parsed_df_3 = pd.json_normalize(data_3['model_response_3_parsed']).drop(["label"],axis=1)
model_response_parsed_df_3.head()

,Label,Headline
0,Business,New York Divorce Law Set for a Change: A Push ...
1,Business,Memos Warned of Billing Fraud by Firm in Iraq:...
2,World,"New Chechen Leader Vows Peace, Poll Criticized"
3,World,Two UN Workers Kidnapped in Afghanistan Call H...
4,World,U.S. State Dept Finishes Review of Iraq Aid Pl...


In [ ]:
data_with_parsed_model_output_3 = pd.concat([data_3, model_response_parsed_df_3], axis=1)
data_with_parsed_model_output_3.head()

,Article,model_response_3,model_response_3_parsed,Label,Headline
0,A New Push to Loosen New York's Divorce Law La...,Sure! Here's my analysis of the article you p...,"{'Label': 'Business', 'Headline': 'New York Di...",Business,New York Divorce Law Set for a Change: A Push ...
1,Memos Warned of Billing Fraud by Firm in Iraq ...,"{""Label"": ""Business"",""Headline"": ""Memos Warned...","{'Label': 'Business', 'Headline': 'Memos Warne...",Business,Memos Warned of Billing Fraud by Firm in Iraq:...
2,"New Chechen Leader Vows Peace, Poll Criticized...","{""Label"": ""World"",""Headline"": ""New Chechen Lea...","{'Label': 'World', 'Headline': 'New Chechen Le...",World,"New Chechen Leader Vows Peace, Poll Criticized"
3,WWW KOTV.com _ Two of three UN workers kidnapp...,"{ ""Label"": ""World"", ""Headline"": ""...","{'Label': 'World', 'Headline': 'Two UN Workers...",World,Two UN Workers Kidnapped in Afghanistan Call H...
4,U.S. State Dept Finishes Review of Iraq Aid Pl...,"{""Label"": ""World"",""Headline"": ""U.S. State Dept...","{'Label': 'World', 'Headline': 'U.S. State Dep...",World,U.S. State Dept Finishes Review of Iraq Aid Pl...


In [ ]:
final_data_3 = data_with_parsed_model_output_3.drop(['model_response_3','model_response_3_parsed'], axis=1)
final_data_3.head()

,Article,Label,Headline
0,A New Push to Loosen New York's Divorce Law La...,Business,New York Divorce Law Set for a Change: A Push ...
1,Memos Warned of Billing Fraud by Firm in Iraq ...,Business,Memos Warned of Billing Fraud by Firm in Iraq:...
2,"New Chechen Leader Vows Peace, Poll Criticized...",World,"New Chechen Leader Vows Peace, Poll Criticized"
3,WWW KOTV.com _ Two of three UN workers kidnapp...,World,Two UN Workers Kidnapped in Afghanistan Call H...
4,U.S. State Dept Finishes Review of Iraq Aid Pl...,World,U.S. State Dept Finishes Review of Iraq Aid Pl...


## Classifying the news articles, generating a headline, and generating a summary

In [ ]:
data_4 = data.copy()

In [ ]:
# defining the instructions for the model
instruction_4 = """
    You are an AI analyzing news articles. Tag the given article using one or more of the below mentioned categories only depending upon the content of the article:
    - World
    - Sports
    - Business
    - Sci/Tech

    Once done,craft a compelling headline summarizing the key insights and significance of the article.Ensure the headline is attention-grabbing and concise while accurately reflecting the content and output it.

    Further,"Summarize the key points and main arguments presented in the news article, while providing a concise yet comprehensive overview of the information and its implications."
    Provide the output in a JSON format with the following keys:
    {
        "Label": "your_label_prediction",
        "Headline": "your_headline_prediction",
        "Summary": "your_summary_prediction"
    }

    Only return the JSON, do not return any other information and remove the extra spaces.
"""

In [ ]:
data_4['model_response_4'] = data_4['Article'].apply(lambda x: generate_llama_response(instruction_4, x).replace('\n', ''))

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.gene

In [ ]:
data_4["model_response_4_parsed"] = data_4["model_response_4"].apply(extract_json_data)

Error parsing JSON: Invalid \escape: line 1 column 126 (char 125)


In [ ]:
data_4["model_response_4_parsed"]

0     {'Label': 'Business', 'Headline': 'New York's ...
1     {'Label': 'Business', 'Headline': 'Memos Warne...
2     {'Label': 'World', 'Headline': 'New Chechen Le...
3     {'Label': 'World', 'Headline': 'Two UN Workers...
4     {'Label': 'World', 'Headline': 'U.S. State Dep...
                            ...                        
95    {'Label': 'Sci/Tech', 'Headline': 'Scientists ...
96    {'Label': 'Sci/Tech', 'Headline': 'Ninth-Grade...
97    {'Label': 'World', 'Headline': 'Congress Wants...
98    {'Label': 'Business', 'Headline': 'IBM Jumps i...
99    {'Label': 'Business', 'Headline': 'Clearing Th...
Name: model_response_4_parsed, Length: 100, dtype: object

In [ ]:
data_4[data_4["model_response_4_parsed"]=={}]

,Article,model_response_4,model_response_4_parsed
43,Hewitt wins Washington Open Australia #39;s Ll...,"{""Label"": ""Sports"",""Headline"": ""Hewitt Wins Wa...",{}


In [ ]:
data_4["model_response_4"][43]

'{"Label": "Sports","Headline": "Hewitt Wins Washington Open, Fine-Tunes Game for US Open","Summary": "Lleyton Hewitt won the \\$500,000 Washington Open, defeating Gilles Muller in straight sets. The victory fine-tuned his game ahead of the US Open."}'

In [ ]:
data_4["model_response_4_parsed"][43] = {"Label": "Sports","Headline": "Hewitt Wins Washington Open, Fine-Tunes Game for US Open","Summary": "Lleyton Hewitt won the $500,000 Washington Open, defeating Gilles Muller in straight sets.The victory fine-tuned his game ahead of the US Open."}

In [ ]:
model_response_parsed_df_4 = pd.json_normalize(data_4['model_response_4_parsed'])
model_response_parsed_df_4.head()

,Label,Headline,Summary
0,Business,New York's Divorce Law Under Scrutiny: A Push ...,Lawyers and judges are advocating for changes ...
1,Business,Memos Warned of Billing Fraud by Firm in Iraq,The article reveals memos that warned of billi...
2,World,"New Chechen Leader Vows Peace, Poll Criticized",Chechnya's new leader vowed to rebuild the reg...
3,World,Two UN Workers Kidnapped in Afghanistan Call H...,Two of three United Nations workers kidnapped ...
4,World,U.S. State Dept Finishes Review of Iraq Aid Plan,The U.S. State Department finished an intensiv...


In [ ]:
data_with_parsed_model_output_4 = pd.concat([data_4, model_response_parsed_df_4], axis=1)
data_with_parsed_model_output_4.head()

,Article,model_response_4,model_response_4_parsed,Label,Headline,Summary
0,A New Push to Loosen New York's Divorce Law La...,"{""Label"": ""Business"",""Headline"": ""New York's D...","{'Label': 'Business', 'Headline': 'New York's ...",Business,New York's Divorce Law Under Scrutiny: A Push ...,Lawyers and judges are advocating for changes ...
1,Memos Warned of Billing Fraud by Firm in Iraq ...,"{""Label"": ""Business"",""Headline"": ""Memos Warned...","{'Label': 'Business', 'Headline': 'Memos Warne...",Business,Memos Warned of Billing Fraud by Firm in Iraq,The article reveals memos that warned of billi...
2,"New Chechen Leader Vows Peace, Poll Criticized...","{ ""Label"": ""World"", ""Headline"":...","{'Label': 'World', 'Headline': 'New Chechen Le...",World,"New Chechen Leader Vows Peace, Poll Criticized",Chechnya's new leader vowed to rebuild the reg...
3,WWW KOTV.com _ Two of three UN workers kidnapp...,"{""Label"": ""World"",""Headline"": ""Two UN Workers ...","{'Label': 'World', 'Headline': 'Two UN Workers...",World,Two UN Workers Kidnapped in Afghanistan Call H...,Two of three United Nations workers kidnapped ...
4,U.S. State Dept Finishes Review of Iraq Aid Pl...,"{ ""Label"": ""World"", ""Headline"":...","{'Label': 'World', 'Headline': 'U.S. State Dep...",World,U.S. State Dept Finishes Review of Iraq Aid Plan,The U.S. State Department finished an intensiv...


In [ ]:
final_data_4 = data_with_parsed_model_output_4.drop(['model_response_4','model_response_4_parsed'], axis=1)
final_data_4.head()

,Article,Label,Headline,Summary
0,A New Push to Loosen New York's Divorce Law La...,Business,New York's Divorce Law Under Scrutiny: A Push ...,Lawyers and judges are advocating for changes ...
1,Memos Warned of Billing Fraud by Firm in Iraq ...,Business,Memos Warned of Billing Fraud by Firm in Iraq,The article reveals memos that warned of billi...
2,"New Chechen Leader Vows Peace, Poll Criticized...",World,"New Chechen Leader Vows Peace, Poll Criticized",Chechnya's new leader vowed to rebuild the reg...
3,WWW KOTV.com _ Two of three UN workers kidnapp...,World,Two UN Workers Kidnapped in Afghanistan Call H...,Two of three United Nations workers kidnapped ...
4,U.S. State Dept Finishes Review of Iraq Aid Pl...,World,U.S. State Dept Finishes Review of Iraq Aid Plan,The U.S. State Department finished an intensiv...


## Conclusion

- We used an LLM to do multiple tasks, one stage at a time
    1. We first identified the category of the news article.
    2. Next, in addition to identifying the category, we generated a headline for it.
    3. Finally, in addition to identifying the category and generating a headline, we also generated a summary for the article.

- To try and improve the model output, one can try the following:
    1. Update the prompt
    2. Update the model parameters (`temparature`, `top_p`, ...)



<font size=6 color='blue'>Power Ahead</font>
___